Test data distribution

In [1]:
import numpy as np
import torch
from data.dataset import DatasetLoader, OneDDatasetLoader
from data.data import TorchGraphData
from preprocessing.normalize import *
from preprocessing.batching_v2 import get_batch_graphs
import math
from typing import List

def flowrate_bc_sin(flowrate, n_edge : int, 
                    n_time : int)->torch.Tensor:
    _flowrate = []
    T = 4.8 # seconds
    for i in range(n_time):
        t = i * T / (n_time - 1)
        value = flowrate(t)
        _flowrate.append(torch.full(size=(n_edge, 1), fill_value=value))
    return torch.cat(_flowrate, dim=1)
        

def normalize(data, mean_ea, std_ea):
    x = min_max_scaler(data.x, min=0, max=500)
    edge_index = data.edge_index
    edge_attr = standard_scaler(data.edge_attr,
                mean=mean_ea,
                std=std_ea)
    pressure = data.pressure*1e-3
    velocity = data.velocity*1e-3
    f = lambda t : (np.pi * 1e0 / 4.8) * math.sin(2 * np.pi * t / 4.8)
    flowrate_bc = flowrate_bc_sin(flowrate=f, n_edge=data.edge_attr.size(0),\
                                n_time=data.flowrate.size(1))
    return TorchGraphData(x=x,edge_index=edge_index,edge_attr=edge_attr,
                        pressure=pressure, velocity=velocity, 
                        flowrate_bc=flowrate_bc)

def train_eval_split(dataset : DatasetLoader, train_id : List, eval_id : List,
                    batch_size : int, batch_n_times : int, recursive : bool):
    mean, std = mean_std_dataset(dataset, set_id=list(range(dataset.len())))
    # Train dataset
    train_dataset = []
    for i in train_id:
        data = normalize(dataset[i], mean, std)
        train_dataset += get_batch_graphs(data=data, batch_size=batch_size,
                        batch_n_times=batch_n_times, recursive=recursive)
    # Test dataset
    eval_dataset = []
    for i in eval_id:
        data = normalize(dataset[i], mean, std)
        eval_dataset += get_batch_graphs(data=data, batch_size=batch_size,
                        batch_n_times=batch_n_times, recursive=recursive)
        # eval_dataset += [data]

    return train_dataset, eval_dataset
def mean_std_dataset(dataset : DatasetLoader, set_id : List, field : str = 'edge_attr'):
    fields = []
    for i in set_id:
        fields.append(dataset[i]._store[field])
    fields = torch.cat(fields, dim=0)
    return fields.mean(axis=0), fields.std(axis=0)

_dataset = OneDDatasetLoader(
    root_dir='/data1/tam/downloaded_datasets_transformed'
)

In [2]:
batched_dataset = _dataset.batching(batch_size=2000, batch_n_times=10, recursive=True,
                                    sub_dir='/batched/')
# batched_dataset = OneDDatasetLoader(
#     root_dir='/data1/tam/downloaded_datasets_transformed',
#     sub_dir='/batched/'
# )

/data1/tam/downloaded_datasets_transformed/batched/batched_data_0.pt
/data1/tam/downloaded_datasets_transformed/batched/batched_data_1.pt
/data1/tam/downloaded_datasets_transformed/batched/batched_data_2.pt
/data1/tam/downloaded_datasets_transformed/batched/batched_data_3.pt
/data1/tam/downloaded_datasets_transformed/batched/batched_data_4.pt
/data1/tam/downloaded_datasets_transformed/batched/batched_data_5.pt
/data1/tam/downloaded_datasets_transformed/batched/batched_data_6.pt
/data1/tam/downloaded_datasets_transformed/batched/batched_data_7.pt
/data1/tam/downloaded_datasets_transformed/batched/batched_data_8.pt
/data1/tam/downloaded_datasets_transformed/batched/batched_data_9.pt
/data1/tam/downloaded_datasets_transformed/batched/batched_data_10.pt
/data1/tam/downloaded_datasets_transformed/batched/batched_data_11.pt
/data1/tam/downloaded_datasets_transformed/batched/batched_data_12.pt
/data1/tam/downloaded_datasets_transformed/batched/batched_data_13.pt
/data1/tam/downloaded_datasets

In [3]:
batched_dataset[10000]

TorchGraphData(x=[2028, 3], edge_index=[2, 2026], edge_attr=[2026, 7], pressure=[2028, 11], flowrate=[2026, 11], velocity=[2026, 11])

In [ ]:
p_min = _dataset.min(var_name='pressure', axis=0).numpy()
p_max = _dataset.max(var_name='pressure', axis=0).numpy()
import matplotlib.pyplot as plt
x = range(p_min.shape[0])
plt.plot(x, p_min, c='red')
plt.plot(x, p_max, c='blue')
plt.show()

In [ ]:
import os
import torch
from networks.network_recurrent import RecurrentMeshGraphNet
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from networks.network_recurrent import objectview
CUDA_LAUNCH_BLOCKING=1
torch.cuda.empty_cache()

# Model params
args = objectview({
    'input_dim_node' : 1,
    'input_dim_edge' : _dataset[0].edge_attr.size(1)+2,
    'output_dim_node' : 1,
    'output_dim_edge' : 1,
    'hidden_dim' : 128,
    'n_processors' : 10,
    'n_time' : _dataset[0].pressure.size(1),
    'device' : torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    # 'device' : torch.device("cpu"),
    'lr' : 1e-7,
    'weight_decay' : 5e-2,
    'epoch' : 100
})

# Model initializing
model1 = RecurrentMeshGraphNet(
    input_dim_node = args.input_dim_node,
    input_dim_edge = args.input_dim_edge,
    output_dim_node = args.output_dim_node,
    output_dim_edge = args.output_dim_edge,
    hidden_dim = args.hidden_dim,
    n_processors = args.n_processors
)
model1 = model1.to(args.device)
model1.load_state_dict(torch.load('models/rmgn_v2_final.pth'))
model2 = RecurrentMeshGraphNet(
    input_dim_node = _dataset[0].x.size(1)+1,
    input_dim_edge = args.input_dim_edge,
    output_dim_node = args.output_dim_node,
    output_dim_edge = args.output_dim_edge,
    hidden_dim = args.hidden_dim,
    n_processors = args.n_processors
)
model2 = model2.to(args.device)
model2.load_state_dict(torch.load('models/rmgn_v1_final.pth'))
models = [model1, model2]
# optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
# criterion = torch.nn.MSELoss()

In [ ]:
# Plot prediction/ground truth
import matplotlib.pyplot as plt

def plot_comparison(models, data, index, var_name = 'Pressure'):
    n_edge = data.edge_attr.size(0)
    hidden = torch.zeros(n_edge, args.hidden_dim).to(args.device)
    _x = data.x
    edge_index = data.edge_index.to(args.device)
    _edge_attr = data.edge_attr

    y_preds = []
    count = 0
    for model in models:
        total_loss = 0
        total_node_out = [data.pressure[:,0].unsqueeze(1)]
        total_edge_out = [data.velocity[:,0].unsqueeze(1)]
        for i in range(1, args.n_time):
            model.eval()
            with torch.no_grad():
                _pressure = data.pressure[:,i-1].unsqueeze(1)
                _velocity = data.velocity[:,i-1].unsqueeze(1)
                _flowrate_bc = data.flowrate_bc[:,i].unsqueeze(1)
                if count == 0:
                    x = torch.cat([_pressure], dim=1).to(args.device)
                elif count == 1:
                    x = torch.cat([_x, _pressure], dim=1).to(args.device)
        
                edge_attr = torch.cat([_edge_attr, _velocity, _flowrate_bc], dim=1).to(args.device)
                node_out, edge_out, hidden = model(x, edge_index, edge_attr, hidden)
                hidden = hidden.detach()

                total_node_out.append(node_out.detach().cpu())
                total_edge_out.append(edge_out.detach().cpu())
        total_node_out = torch.cat(total_node_out, dim=1)
        total_edge_out = torch.cat(total_edge_out, dim=1)
        if var_name == 'Pressure':
            y_preds.append(total_node_out[index].numpy())
        elif var_name == 'Velocity':
            y_preds.append(total_edge_out[index].numpy())

        count += 1
    
    color = ['red', 'green', 'blue', 'black']
    # plot
    if var_name == 'Pressure':
        y_true = data.pressure[index].numpy()
    elif var_name == 'Velocity':
        y_true = data.velocity[index].numpy()
    x = [i * 4.8 /200 for i in range(201)]
    for i in range(len(models)):
        plt.plot(x, y_preds[i], c=color[i], label='RMGNv'+str(i))
    plt.plot(x, y_true, c=color[-1], linestyle='dashdot', label='ground_truth')
    # plt.ylim([-50,50])
    plt.legend(loc='upper right')
    plt.ylabel(var_name, fontsize=20)
    plt.xlabel('Time', fontsize=20)
    plt.show()
    
    return total_loss
    
mean, std = mean_std_dataset(_dataset, set_id=list(range(_dataset.len())))
plot_comparison(models, normalize(_dataset[40], mean, std), index=20, var_name='Velocity')